In [1]:
import cv2
import numpy as np
import time

# Parametros para lk_track

# problemas actuales:
*detectar cual marcador corresponde con el siguiente en la deteccion

*analizar hacia adonde se dirige el flujo

In [4]:
#Importar video

fileName='PC124surfTrim203-2.mp4'
cap = cv2.VideoCapture(fileName)

cantidadFrames=50
#por que se usa un tipo de video sin ningun tipo de estabilizacion, 
#el codigo funciona mejor con una cantidad muy limitada de frames


skipframes=400
#saltea una cantidad x de frames al inicio


mask=0
anguloprom=0
moduloprom=0

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))

#numero de frames que saltea al inicio
#while(skipframes):
#    cap.read()
#    skipframes-=1

while(1):
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    puntos=np.zeros([p0.shape[0],cantidadFrames+2,2])#arreglo de puntos para analizar

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    cvtexto= np.zeros_like(old_frame)
    #actualFrame=0


    for actualFrame in range(0,5):
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]


        #anaizar correspodencia entre puntos buenos viejos y nuevos
        if actualFrame==1:
            for a in range (0,good_old.shape[0]):#optimizar
                puntos[a,0,:]=good_old[a]

        cantidadPuntosNuevos,v =good_new.shape
        for pos in range (0, cantidadPuntosNuevos):
            for pospuntos in range (0,puntos.shape[0]):
                x=puntos[pospuntos,actualFrame-1,0]-good_new[pos,0]#distancia en x
                y=puntos[pospuntos,actualFrame-1,1]-good_new[pos,1]#distancia en y
                dist=np.linalg.norm([x,y])#distancia entre los 2 puntos
                if dist<10:#si la distancia es menor a 10 "pixeles" coloca el punto como coincidente con el anterior
                    puntos[pospuntos,actualFrame]=good_new[pos]

                    #if puntos[pospuntos,actualFrame]==[0,0]:
                    #    puntos[pospuntos,actualFrame]=good_new[pos]
                    #else:
                    #    comparar cual tiene distancia minima con el punto anterior


        #time.sleep(.5)#pausa para ver el programa

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
            
        img = cv2.add(frame,mask)
        imgflecha= cv2.arrowedLine(img,
                                   (100,100),
                                   ((100+10*moduloprom*np.cos(np.radians(anguloprom+180))).astype(int),
                                    (100+10*moduloprom*np.sin(np.radians(anguloprom+180))).astype(int)),
                                   (0,0,255),
                                    4)#añade flacha de modulo y fase
        
        
        imgtexto= cv2.putText(imgflecha,"Modulo: "+str(moduloprom), (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)#añade texto con modulo
        imgtexto2= cv2.putText(imgtexto,"Angulo"+str(anguloprom), (10,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)#añade texto con modulo
        
        cv2.imshow('frame',imgtexto2)
        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)


    #limpia ordena los puntos y quita las filas con solo 0
    puntosLimp=puntos[:,~np.all(puntos == 0, axis=0)]
    puntosLimp2=puntosLimp[~np.all(puntosLimp == 0, axis=1)]
    puntosLimp2.shape#los ordeno en un arreglo de 2 dimension para poder guardar el archivo mas facil

    #calculo del angulo y modulo de desplazamiento de los puntos
    angulo=np.zeros(puntosLimp2.shape[0])
    modulo=np.zeros(puntosLimp2.shape[0])

    for pos in range (2, puntosLimp2.shape[0]):
        contador=0
        mod=0
        ang=0
        for elemento in range (2,puntosLimp2.shape[1],2):
            if puntosLimp2[pos,elemento]!=0:
                contador+=1
                x=puntosLimp2[pos,elemento-2]-puntosLimp2[pos,elemento]
                y=puntosLimp2[pos,elemento-1]-puntosLimp2[pos,elemento+1]
                mod+=np.linalg.norm([x,y])#distancia entre los 2 puntos
                ang+=np.angle(x+y*1j,deg=True)

        if contador!=0:
            modulo[pos]=mod/contador
            angulo[pos]=ang/contador

    #obtengo los mismos modulos y angulo con diferentes cantidades de muestras de frames para el mismo momento en el video

    anguloprom=np.average(angulo)
    moduloprom=np.average(modulo)
    # Calcular errores medios cuadraticos


    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)


    
cv2.destroyAllWindows()
cap.release()

error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\color.cpp:11048: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


<img src="vlcsnap-2018-06-13-15h54m57s177.PNG">

In [ ]:
segmentar por luminancia
hough  HUE
ver longitud de onda